In [10]:
# ✅ ETF 與台積電報酬率完整批次分析專題版

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import numpy as np
import platform
import os


In [ ]:
import os
import certifi
os.environ['SSL_CERT_FILE'] = certifi.where()

#安裝 certifi
#pip install certifi

In [13]:
# ✅ 字體設定（支援跨平台中文）
if platform.system() == 'Darwin':
    plt.rcParams['font.family'] = ['Arial Unicode MS']
elif platform.system() == 'Windows':
    plt.rcParams['font.family'] = ['Microsoft JhengHei']
else:
    plt.rcParams['font.family'] = ['Noto Sans CJK TC']

plt.rcParams['axes.unicode_minus'] = False


In [14]:

# ✅ 資料參數設定
base_stock_symbol = "2330.TW"
base_stock_name = "台積電"


In [15]:

# ✅ 你可以在這裡放入你要分析的多個 ETF
target_list = [
    {"symbol": "0050.TW", "name": "0050元大台灣50ETF"},
    {"symbol": "0056.TW", "name": "0056高股息ETF"},
    {"symbol": "00878.TW", "name": "00878高股息ETF"},
    {"symbol": "00692.TW", "name": "富邦公司治理ETF"}
]

start_date = "2022-01-01"
# end_date = "2024-12-31"
end_date = "2025-4-30"


In [16]:

# ✅ 建立儲存資料夾
os.makedirs("ETF分析圖表", exist_ok=True)


In [17]:

# ✅ 先下載台積電資料 (只下載一次)
base_data = yf.download(base_stock_symbol, start=start_date, end=end_date)
base_data['Return'] = base_data['Close'].pct_change()


Failed to get ticker '2330.TW' reason: Failed to perform, curl: (60) SSL certificate problem: unable to get local issuer certificate. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['2330.TW']: YFTzMissingError('possibly delisted; no timezone found')


In [11]:

# ✅ 逐一分析每個目標ETF

results = []  # 儲存每次分析結果

for target in target_list:
    compare_stock_symbol = target["symbol"]
    compare_stock_name = target["name"]

    print(f"\n開始分析 {compare_stock_name}...")

    # 下載ETF資料
    compare_data = yf.download(compare_stock_symbol, start=start_date, end=end_date)
    compare_data['Return'] = compare_data['Close'].pct_change()

    # 合併報酬率
    returns = pd.DataFrame({
        f'{base_stock_symbol}_Return': base_data['Return'],
        f'{compare_stock_symbol}_Return': compare_data['Return']
    }).dropna()

    # 線性回歸
    X = returns[f'{base_stock_symbol}_Return'].values.reshape(-1, 1)
    y = returns[f'{compare_stock_symbol}_Return'].values

    model = LinearRegression()
    model.fit(X, y)

    slope = model.coef_[0]
    intercept = model.intercept_

    # 畫圖
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=returns[f'{base_stock_symbol}_Return'], y=returns[f'{compare_stock_symbol}_Return'], alpha=0.5, label='報酬率點')
    plt.plot(returns[f'{base_stock_symbol}_Return'], model.predict(X), color='red', label=f'趨勢線：y = {slope:.2f}x + {intercept:.4f}')
    plt.title(f'{base_stock_name} 與 {compare_stock_name} 報酬率散佈圖與回歸趨勢線')
    plt.xlabel(f'{base_stock_name} 報酬率')
    plt.ylabel(f'{compare_stock_name} 報酬率')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    filename = f"ETF分析圖表/{compare_stock_symbol}_回歸分析圖.png"
    plt.savefig(filename)
    plt.close()

    # 儲存結果
    results.append({
        "ETF代碼": compare_stock_symbol,
        "ETF名稱": compare_stock_name,
        "斜率": slope,
        "截距": intercept
    })



開始分析 0050元大台灣50ETF...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


開始分析 0056高股息ETF...



[*********************100%***********************]  1 of 1 completed


開始分析 00878高股息ETF...



[*********************100%***********************]  1 of 1 completed


開始分析 富邦公司治理ETF...


In [12]:

# ✅ 匯出整體回歸結果報表
result_df = pd.DataFrame(results)
result_df.to_csv("ETF分析圖表/回歸分析總表.csv", index=False)

print("\n所有分析已完成，結果已儲存在 'ETF分析圖表/' 資料夾內。")
display(result_df)


所有分析已完成，結果已儲存在 'ETF分析圖表/' 資料夾內。


,ETF代碼,ETF名稱,斜率,截距
0,0050.TW,0050元大台灣50ETF,0.677608,-0.000086
1,0056.TW,0056高股息ETF,0.359009,0.000141
2,00878.TW,00878高股息ETF,0.329642,0.000189
3,00692.TW,富邦公司治理ETF,0.602051,-0.000054
